In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
symbol = 'BTC/USDT'
timeframe = '1h'
ma_short_period = 20
ma_long_period = 50
balance_percentage = 0.1

In [10]:
data = pd.read_csv(r"C:\Users\karim\OneDrive\Bureau\Data Engineer\DS Project\Cryptobot\DataBase\1h\BTC-USDT-USDT.csv")
data.index = pd.to_datetime(data['date'], unit='ms')


,date,open,high,low,close,volume
date,,,,,,
2019-07-10 11:00:00,1562756400000,13073.5,13075.5,13044.5,13064.5,4.8952
2019-07-10 12:00:00,1562760000000,13064.5,13064.5,12900.5,13029.0,684.5329
2019-07-10 13:00:00,1562763600000,13029.0,13127.5,12996.5,13039.5,812.2584
2019-07-10 14:00:00,1562767200000,13039.5,13065.5,12549.5,12549.5,679.7040
2019-07-10 15:00:00,1562770800000,12549.5,12552.0,12121.0,12362.0,715.6550
...,...,...,...,...,...,...
2024-04-19 23:00:00,1713567600000,63964.2,64095.3,62890.7,63771.2,14875.2400
2024-04-20 00:00:00,1713571200000,63771.2,64216.4,63365.5,63401.3,8151.1100
2024-04-20 01:00:00,1713574800000,63401.3,63741.9,63031.2,63606.8,7683.9050


In [ ]:
def calculate_indicators(data):
    data['ma_short'] = data['close'].rolling(window=ma_short_period).mean()
    data['ma_long'] = data['close'].rolling(window=ma_long_period).mean()
    return data

In [ ]:
def trading_strategy(data):
    if data['ma_short'].iloc[-1] > data['ma_long'].iloc[-1] and data['ma_short'].iloc[-2] <= data['ma_long'].iloc[-2]:
        return 'buy'
    elif data['ma_short'].iloc[-1] < data['ma_long'].iloc[-1] and data['ma_short'].iloc[-2] >= data['ma_long'].iloc[-2]:
        return 'sell'
    else:
        return 'hold'

In [ ]:
# Fonction principale de trading
def main():
    while True:
        data = fetch_ohlcv(symbol, timeframe)
        data = calculate_indicators(data)
        signal = trading_strategy(data)

        
        usdt_balance = 1000
        btc_balance = 0
        taker_fee=0.01
        maker_fee=0.005

        if signal == 'buy' and usdt_balance > 10:
            amount_to_buy = usdt_balance * balance_percentage / data['close'].iloc[-1]
            binance.create_market_buy_order(symbol, amount_to_buy)
            print(f'Bought {amount_to_buy} BTC')

        elif signal == 'sell' and btc_balance > 0.001:
            binance.create_market_sell_order(symbol, btc_balance * balance_percentage)
            print(f'Sold {btc_balance * balance_percentage} BTC')

        time.sleep(3600)  # Attendre une heure avant le prochain trade
